In [1]:
from pymatgen.core import Structure
from joblib import Memory
from mp_api.client import MPRester
from ase.visualize import view

memory = Memory('.cachedir')

In [2]:
def view_structure(structure, viewer='ase'):
    return view(structure.to_ase_atoms(), viewer=viewer)


In [3]:
@memory.cache
def get_structure(mp_id):
    with MPRester() as mpr:
        # docs = mpr.materials.summary.search(material_ids=[mp_id], fields=["structure"])
        # structure = docs[0].structure
        # # -- Shortcut for a single Materials Project ID:
        structure = mpr.get_structure_by_material_id(mp_id,)
    
    return structure

structure = get_structure('mp-1265')
view_structure(structure)

<Popen: returncode: None args: ['/Users/tw/miniforge3/envs/surfpes/bin/pytho...>

In [12]:
from pymatgen.core.surface import generate_all_slabs
# from mp_api.client import MPRester


# structure = get_structure('mp-1265')
slabs = generate_all_slabs(structure.to_conventional(), 1, 10, 20, include_reconstructions=False, primitive=True, max_normal_search=6,)
# there is an apparent mismatch between orientated unit cell and the slab if primitive=True, max_normal_search=None
# this doesn't occur if primitive=True, max_normal_search=5
# What is happening? Not sure!
# /Users/tw/miniforge3/envs/surfpes/lib/python3.11/site-packages/pymatgen/core/surface.py#1156
print(len(slabs))

3


In [13]:
import ipywidgets as widgets
from IPython.display import clear_output

def show_slab(idx):
    global i
    i = idx
    clear_output(wait=True)
    print(slabs[i].reconstruction)
    print(slabs[i])
    return view_structure(slabs[i] * (2, 2, 1), viewer='x3d')

widgets.interact(
    show_slab,
    idx=widgets.IntSlider(value=0, min=0, max=len(slabs)-1, step=1, description="i")
);

interactive(children=(IntSlider(value=0, description='i', max=2), Output()), _dom_classes=('widget-interact',)…

## Lattice Matching

In [18]:
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer

In [22]:
with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        # elements=[element]
        material_ids=["mp-149", "mp-13", "mp-22526"]
    )

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [60]:
from pymatgen.core import Element
from pathlib import Path
from monty.serialization import loadfn, dumpfn

In [96]:
mp_data_path = Path('mp_data'); mp_data_path.mkdir(exist_ok=True)

def get_elemental_data():
    element_to_summary = {}

    for i in range(1, 86):
        el = Element.from_Z(i)
        el_path = mp_data_path/str(el); el_path.mkdir(exist_ok=True)

        if el.is_metal:
            el = str(el)
            summary_path = el_path/'summary.json.gz'
            if summary_path.exists():
                element_to_summary[el] = loadfn(summary_path)
            else:
                with MPRester() as mpr:
                    docs = mpr.materials.summary.search(
                        elements=[el],
                        num_elements=1,
                        is_stable=True,
                    )
                if len(docs) == 0:
                    with MPRester() as mpr:
                        docs = sorted(mpr.materials.summary.search(
                            elements=[el],
                            num_elements=1,
                            is_stable=False,
                        ), key=lambda d: d.formation_energy_per_atom)
                        docs = [docs[0]]
                assert len(docs) == 1, f"Expected 1 summary document for {el}, got {len(docs)}"
                dumpfn(docs[0].dict(), summary_path)
                element_to_summary[el] = docs[0]
    return element_to_summary

def get_elasticity_data(element_to_summary):
    element_to_elasticity = {}
    for el, summary in element_to_summary.items():
        elastic_path = mp_data_path/str(el)/'elastic.json.gz'
        if elastic_path.exists():
            element_to_elasticity[el] = loadfn(elastic_path)
        else:
            with MPRester() as mpr:
                docs = mpr.materials.elasticity.search(material_ids=[summary.get('material_id')],)
            if len(docs) == 0:
                print(f"No elasticity data for {el} ({summary['material_id']})")
                continue
                
            assert len(docs) == 1, f"Expected 1 elasticity document for {el} ({summary['material_id']}), got {len(docs)}"
            dumpfn(docs[0].dict(), elastic_path)
            element_to_elasticity[el] = docs[0]
    return element_to_elasticity
 

el_to_summary = get_elemental_data()      
el_to_elasticity = get_elasticity_data(el_to_summary)      

Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

No elasticity data for K (mp-1184804)


Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

No elasticity data for Mn (mp-35)


Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

...

No elasticity data for Eu (mp-21462)


Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

No elasticity data for Er (mp-1184115)


Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

No elasticity data for Yb (mp-1187875)


## Finding Substrates

In [145]:
el_to_elasticity['Mg']['elastic_tensor'].keys()

dict_keys(['raw', 'ieee_format'])

In [151]:
from pymatgen.analysis.elasticity.elastic import ElasticTensor

sa = SubstrateAnalyzer(1, 1)
mg_elastic_tensor = ElasticTensor.from_voigt(el_to_elasticity['Mg']['elastic_tensor']['raw'])

In [152]:
for match in sa.calculate(
    el_to_summary['Pt']['structure'],
    el_to_summary['Mg']['structure'],
    mg_elastic_tensor,
    lowest=True,
 ):
    print(match)

SubstrateMatch(film_sl_vectors=array([[ 1.60978358,  4.55315908,  2.78822888],
       [ 4.02446095, -2.27657954,  6.97057016]]), substrate_sl_vectors=array([[-1.00000000e-06, -5.49410482e+00,  0.00000000e+00],
       [ 7.93005880e+00, -2.74705241e+00, -0.00000000e+00]]), film_vectors=(array([-0.80489179, -2.27657954, -1.39411444]), array([ 1.60978458, -2.27657954,  2.78822786])), substrate_vectors=(array([ 1.58601136, -2.74705241, -0.        ]), array([-1.58601236, -2.74705241, -0.        ])), film_transformation=array([[1., 2.],
       [0., 4.]]), substrate_transformation=array([[1., 1.],
       [0., 5.]]), film_miller=(1, 0, -1), substrate_miller=(0, 0, 1), strain=Strain([[-7.32946172e-03 -4.28494659e-08 -7.32945930e-03]
 [-4.28494659e-08  5.56405599e-03 -4.29853383e-08]
 [-7.32945930e-03 -4.29853383e-08 -7.32945687e-03]]), von_mises_strain=np.float64(0.012062902861236449), ground_state_energy=0, elastic_energy=0.0006491347880201532)
SubstrateMatch(film_sl_vectors=array([[ -2.4146753

In [154]:
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder

cib = CoherentInterfaceBuilder(
    substrate_structure=el_to_summary['Mg']['structure'],
    film_structure=el_to_summary['Pt']['structure'],
    film_miller=match.film_miller,
    substrate_miller=match.substrate_miller,
)

In [158]:
cib.terminations

[('Pt_R-3m_1', 'Mg_C2/m_1')]

In [ ]:
cib.get_interfaces(termination=)

TypeError: CoherentInterfaceBuilder.get_interfaces() missing 1 required positional argument: 'termination'